# Subtask A

In [53]:
import pandas as pd
import numpy as np

In [54]:
texts_df = pd.read_csv('subtaskA_train.csv')

In [55]:
texts_df.head()

,comment_text,conspiratorial
0,⚡Se non ci fossero soldati non ci sarebbero gu...,0
1,"21/08/21]( [PRE-PRINT]\n\n📄__ ""Shedding of Inf...",1
2,PAURA E DELIRIO ALLA CNN: IL MINISTERO DELLA V...,1
3,L'Aspirina non aumenta la sopravvivenza dei pa...,0
4,L'Italia non puo' dare armi lo vieta la Costit...,0


In [56]:
texts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1845 entries, 0 to 1844
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   comment_text    1842 non-null   object
 1   conspiratorial  1845 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 29.0+ KB


In [57]:
texts_df[texts_df['comment_text'].isna()]

,comment_text,conspiratorial
244,NaN,0
263,NaN,0
665,NaN,0


Delete rows with NaN text

In [58]:
texts_df = texts_df[texts_df.comment_text.notna()]

In [59]:
texts_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1842 entries, 0 to 1844
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   comment_text    1842 non-null   object
 1   conspiratorial  1842 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 43.2+ KB


Count positive and negatie samples

In [60]:
texts_df.conspiratorial.value_counts()

1    925
0    917
Name: conspiratorial, dtype: int64

## Train-Validation split

In [61]:
from sklearn.model_selection import StratifiedShuffleSplit

In [62]:
# Split the data into training and testing datasets using stratified sampling
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, eval_index in split.split(texts_df, texts_df.conspiratorial):
    train_df, eval_df = texts_df.iloc[train_index], texts_df.iloc[eval_index]

In [63]:
print(train_df.info())
print(train_df.conspiratorial.value_counts())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1473 entries, 1512 to 1771
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   comment_text    1473 non-null   object
 1   conspiratorial  1473 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 34.5+ KB
None
1    740
0    733
Name: conspiratorial, dtype: int64


In [64]:
print(eval_df.info())
print(eval_df.conspiratorial.value_counts())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 369 entries, 363 to 670
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   comment_text    369 non-null    object
 1   conspiratorial  369 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 8.6+ KB
None
1    185
0    184
Name: conspiratorial, dtype: int64


The dataset seems balanced in term of positive and negative samples.

## Model

In [65]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

In [66]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [67]:
train_df.columns = ["text", "labels"]
eval_df.columns = ["text", "labels"]

In [68]:
import torch


cuda_available = torch.cuda.is_available()
print(cuda_available)


True


## Model configuration

In [69]:
# other models

# model = ClassificationModel("bert", "dbmdz/bert-base-italian-cased", args=model_args, use_cuda=cuda_available)
# model = ClassificationModel("distilbert", "indigo-ai/BERTino", args=model_args, use_cuda=cuda_available) --- 0.81 with 5 epochs

In [70]:
batch_size = 8

# Num steps in epoch = num training samples / batch size
steps_per_epoch = int(np.ceil(len(train_df) / float(batch_size)))

print('Each epoch will have {:,} steps.'.format(steps_per_epoch))

Each epoch will have 185 steps.


In [71]:
# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=20, do_lower_case=False, evaluate_during_training=True, evaluate_during_training_verbose=True, # Main options
                                use_multiprocessing=False, use_multiprocessing_for_evaluation=False, overwrite_output_dir=True, # System configurations
                                eval_batch_size=batch_size, train_batch_size=batch_size, evaluate_during_training_steps=steps_per_epoch, # Batch sizes and steps
                                use_early_stopping=True, early_stopping_metric='eval_loss', early_stopping_patience=2, early_stopping_metric_minimize=True, # Early stopping
                                early_stopping_delta=0.001, early_stopping_consider_epochs=True
                                   )

# Create a ClassificationModel

model = ClassificationModel("bert", "dbmdz/bert-base-italian-xxl-cased", args=model_args, use_cuda=cuda_available)

Some weights of the model checkpoint at dbmdz/bert-base-italian-xxl-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification w

## Train

In [72]:
# Train the model
model.train_model(train_df, eval_df=eval_df)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_2_2


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 0 of 20:   0%|          | 0/185 [00:00<?, ?it/s]

/home/giacomo/anaconda3/envs/nlp/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_2
INFO:simpletransformers.classification.classification_model:{'mcc': 0.5077451753055425, 'tp': 98, 'tn': 172, 'fp': 12, 'fn': 87, 'auroc': 0.8576674500587544, 'auprc': 0.8684350593246475, 

Running Epoch 1 of 20:   0%|          | 0/185 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_2
INFO:simpletransformers.classification.classification_model:{'mcc': 0.5741834457642736, 'tp': 125, 'tn': 163, 'fp': 21, 'fn': 60, 'auroc': 0.8949471210340775, 'auprc': 0.892329266999466, 'eval_loss': 0.47875352616005756}
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_2
INFO:simpletransformers.classification.classification_model:{'mcc': 0.5741834457642736, 'tp': 125, 'tn': 163, 'fp': 21, 'fn': 60, 'auroc': 0.8949471210340775, 'auprc': 0.892329266999466, 'eval_loss': 0.47875352616005756}
INFO:simpletransformers.classification.classification_model: No improvement in

Running Epoch 2 of 20:   0%|          | 0/185 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_2
INFO:simpletransformers.classification.classification_model:{'mcc': 0.6167792016199172, 'tp': 143, 'tn': 155, 'fp': 29, 'fn': 42, 'auroc': 0.8878231492361927, 'auprc': 0.8908460260651492, 'eval_loss': 0.9220846434856983}
INFO:simpletransformers.classification.classification_model: No improvement in eval_loss
INFO:simpletransformers.classification.classification_model: Current step: 2
INFO:simpletransformers.classification.classification_model: Early stopping patience: 2
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_2
INFO:simpletransformers.classification.classi

(555,
 defaultdict(list,
             {'global_step': [185, 185, 370, 370, 555, 555],
              'train_loss': [1.3037109375,
               1.3037109375,
               1.98046875,
               1.98046875,
               0.0024662017822265625,
               0.0024662017822265625],
              'mcc': [0.5077451753055425,
               0.5077451753055425,
               0.5741834457642736,
               0.5741834457642736,
               0.6167792016199172,
               0.6167792016199172],
              'tp': [98, 98, 125, 125, 143, 143],
              'tn': [172, 172, 163, 163, 155, 155],
              'fp': [12, 12, 21, 21, 29, 29],
              'fn': [87, 87, 60, 60, 42, 42],
              'auroc': [0.8576674500587544,
               0.8576674500587544,
               0.8949471210340775,
               0.8949471210340775,
               0.8878231492361927,
               0.8878231492361927],
              'auprc': [0.8684350593246475,
               0.8684350593246475,


## Evaluate

In [73]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [74]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
print(result)
print(wrong_predictions)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_2


Running Evaluation:   0%|          | 0/47 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.6167792016199172, 'tp': 143, 'tn': 155, 'fp': 29, 'fn': 42, 'auroc': 0.8878231492361927, 'auprc': 0.8908460260651492, 'eval_loss': 0.9220846434856983}


{'mcc': 0.6167792016199172, 'tp': 143, 'tn': 155, 'fp': 29, 'fn': 42, 'auroc': 0.8878231492361927, 'auprc': 0.8908460260651492, 'eval_loss': 0.9220846434856983}
[{'guid': 10, 'text_a': "\nLetters scritta dalla ditta.ssa Gatti a Robert Kennedy jr. Per spiegargli la situazione in Italia oggi.\nC'è sia il testo originale in inglese che la traduzione.", 'text_b': None, 'label': 1}, {'guid': 15, 'text_a': "In breve tempo, per accedere a Internet, sarà richiesto un documento d'identità Digitale.", 'text_b': None, 'label': 1}, {'guid': 18, 'text_a': 'Bisogna proprio dirlo: questi NO-VAX seminano odio ovunque e sono dei teppisti.\n[🇮🇹', 'text_b': None, 'label': 0}, {'guid': 19, 'text_a': '"EBBASTAAA"\n\n', 'text_b': None, 'label': 0}, {'guid': 20, 'text_a': 'Si, penso ai bambini vaccinati e più in generale a tutti quelli che non hanno avuto una reale scelta per un motivo o per un altro, senza entrare nel merito del perché o per come', 'text_b': None, 'label': 1}, {'guid': 31, 'text_a': "Israel

In [75]:
full_pred = model.predict(eval_df.text.tolist())
pred = full_pred[0]
raw_pred = full_pred[1]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/47 [00:00<?, ?it/s]

In [76]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [77]:
# Functions that prints the classification report
def report_scores(test_label, test_pred):
    print(classification_report(test_label, 
                            test_pred, 
                            target_names=['non-conspiratorial', 'conspiratorial']))

In [78]:
report_scores(eval_df.labels.tolist(), pred)

                    precision    recall  f1-score   support

non-conspiratorial       0.79      0.84      0.81       184
    conspiratorial       0.83      0.77      0.80       185

          accuracy                           0.81       369
         macro avg       0.81      0.81      0.81       369
      weighted avg       0.81      0.81      0.81       369

